# Data Extraction and Text Analysis

## Objective 

Objective of this assignment is to extract some sections (which are mentioned below) from SEC / EDGAR financial reports and perform text analysis to compute variables.

## Note

Maintain the following directory structure before running this notebook:
```bash
.
├── LoughranMcDonald_MasterDictionary_2018.csv
├── Output\ Data\ Structure.xlsx
├── cik_list.xlsx
├── constraining_dictionary.xlsx
├── reports
│   ├── mda
│   ├── qqdmr
│   └── rf
├── reports_cleaned
│   ├── mda
│   ├── qqdmr
│   └── rf
├── stopwords
│   ├── StopWords_Auditor.txt
│   ├── StopWords_Currencies.txt
│   ├── StopWords_DatesandNumbers.txt
│   ├── StopWords_Generic.txt
│   ├── StopWords_GenericLong.txt
│   ├── StopWords_Geographic.txt
│   └── StopWords_Names.txt
│   
└── uncertainty_dictionary.xlsx
```

## Data Extraction

In [1]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from collections import Counter
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords as nltk_stopwords
import pyphen

import re
from itertools import chain 
import os
import warnings
warnings.filterwarnings("ignore")

### Reading in filenames

In [2]:
cik_list = pd.read_excel('data/cik_list.xlsx')
cik_list.head(5)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [3]:
cik_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   CIK       152 non-null    int64         
 1   CONAME    152 non-null    object        
 2   FYRMO     152 non-null    int64         
 3   FDATE     152 non-null    datetime64[ns]
 4   FORM      152 non-null    object        
 5   SECFNAME  152 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 7.2+ KB


### Downloading Reports

In [4]:
base_url = "https://www.sec.gov/Archives/"

In [5]:
# Don't run this cell --- unless no data
# downloading each report and saving it

for index, row in cik_list.iterrows():
    r = requests.get(base_url + row['SECFNAME'])
    open('data/reports/' + row['SECFNAME'].split(os.sep)[3], 'wb').write(r.content)

## 1. Sentimental Analysis

### 1.1 Cleaning using Stop Words Lists

In [6]:
# reading stopwords

StopWords_Auditor = pd.read_csv('data/stopwords/StopWords_Auditor.txt', header = None)
StopWords_Currencies = pd.read_fwf('data/stopwords/StopWords_Currencies.txt', header = None)
StopWords_DatesandNumbers = pd.read_csv('data/stopwords/StopWords_DatesandNumbers.txt', header = None)
StopWords_Generic = pd.read_csv('data/stopwords/StopWords_Generic.txt', header = None)
StopWords_GenericLong = pd.read_csv('data/stopwords/StopWords_GenericLong.txt', header = None)
StopWords_Geographic = pd.read_csv('data/stopwords/StopWords_Geographic.txt', header = None)
StopWords_Names = pd.read_csv('data/stopwords/StopWords_Names.txt', header = None)

In [7]:
StopWords_Auditor.head()

,0
0,ERNST
1,YOUNG
2,DELOITTE
3,TOUCHE
4,KPMG


Removing NA values from stopwords currencies.

In [8]:
StopWords_Currencies[~StopWords_Currencies[1].isna()]

,0,1
76,SPECIAL DRAWING RIGHTS | International Monetary,Fund


In [9]:
StopWords_Currencies.iloc[76,1] = StopWords_Currencies.iloc[76,0] + " " + StopWords_Currencies.iloc[76,1]

In [10]:
StopWords_Currencies.drop(1, axis = 1, inplace = True)

In [11]:
StopWords_Currencies.head()

,0
0,AFGHANI | Afghanistan
1,ARIARY | Madagascar
2,BAHT | Thailand
3,BALBOA | Panama
4,BIRR | Ethiopia


Splitting pipe symbols

In [12]:
def split_pipe(x):
    return pd.Series([x.split(" | ")[0].strip(), x.split(" | ")[1].strip()])

Splitting the words in column 0.

In [13]:
StopWords_Currencies[[0, 1]] = StopWords_Currencies[0].apply(lambda x: split_pipe(x))

In [14]:
StopWords_Currencies.head()

,0,1
0,AFGHANI,Afghanistan
1,ARIARY,Madagascar
2,BAHT,Thailand
3,BALBOA,Panama
4,BIRR,Ethiopia


Removing pipes in Dates and Numbers

In [15]:
def remove_pipe(x):
    return x.split(" | ")[0].strip()

In [16]:
StopWords_DatesandNumbers[0] = StopWords_DatesandNumbers[0].apply(lambda x: remove_pipe(x))

In [17]:
StopWords_DatesandNumbers.head()

,0
0,HUNDRED
1,THOUSAND
2,MILLION
3,BILLION
4,TRILLION


In [18]:
StopWords_Generic.head()

,0
0,ABOUT
1,ABOVE
2,AFTER
3,AGAIN
4,ALL


In [19]:
StopWords_GenericLong.head()

,0
0,a
1,a's
2,able
3,about
4,above


Removing pipes in Geographic

In [20]:
StopWords_Geographic[0] = StopWords_Geographic[0].apply(lambda x: remove_pipe(x))

In [21]:
StopWords_Geographic.head()

,0
0,UNITED
1,STATE
2,NORTH
3,SOUTH
4,EAST


Removing pipes from names

In [22]:
StopWords_Names[0] = StopWords_Names[0].astype(str)

In [23]:
StopWords_Names[0] = StopWords_Names[0].apply(lambda x: remove_pipe(x))

In [24]:
StopWords_Names.head()

,0
0,SMITH
1,JOHNSON
2,WILLIAMS
3,JONES
4,BROWN


In [25]:
# listing out all stop words
stopwords = list(chain(StopWords_Currencies[0].to_list(),
                       StopWords_Currencies[1].to_list(),
                       StopWords_DatesandNumbers[0].to_list(),
                       StopWords_Generic[0].to_list(),
                       StopWords_GenericLong[0].to_list(),
                       StopWords_Geographic[0].to_list(),
                       StopWords_Names[0].to_list()))
stopwords_dict = Counter(stopwords) # counter for faster execution

In [26]:
# iterating through each file, removing stop words and saving it

for file in cik_list.SECFNAME:
    f = open('data/reports/' + file.split(os.sep)[3], 'r').read()
    f_clean = ' '.join([word for word in f.split() if word not in stopwords_dict])
    open('data/reports_cleaned/' + file.split(os.sep)[3], 'w').write(f_clean)

### 1.2 Creating dictionary of Positive and Negative words

In [27]:
master_dictionary = pd.read_csv('data/LoughranMcDonald_MasterDictionary_2018.csv')

In [28]:
master_dictionary.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480000e-08,1.240000e-08,3.560000e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.600000e-10,9.730000e-12,9.860000e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.280000e-10,1.390000e-10,6.230000e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.410000e-10,3.160000e-10,9.380000e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.870000e-07,3.680000e-07,3.370000e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [29]:
negative_dict = Counter(master_dictionary['Word'][master_dictionary.Negative > 0].to_list())
positive_dict = Counter(master_dictionary['Word'][master_dictionary.Positive > 0].to_list())
negative_dict = Counter(word for word in negative_dict if word not in stopwords_dict)
positive_dict = Counter(word for word in positive_dict if word not in stopwords_dict)

### 1.3 Extracting Derived variables

In [30]:
# iterating through each file

for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting positive score
    positive_f_dict = doc_dict & positive_dict
    cik_list.loc[i, 'positive_score'] = sum(positive_f_dict.values())
    
    # getting negative score
    negative_f_dict = doc_dict & negative_dict
    cik_list.loc[i, 'negative_score'] = sum(negative_f_dict.values())
    
    # total words after cleaning
    cik_list.loc[i, 'total_words_after_cleaning'] = sum(doc_dict.values())

In [31]:
cik_list['polarity_score'] = cik_list\
.apply(lambda x: (x['positive_score'] - x['negative_score'])/((x['positive_score'] + x['negative_score'])+0.000001),
      axis = 1)

In [32]:
cik_list['subjectivity_score'] = cik_list\
.apply(lambda x: (x['positive_score'] + x['negative_score'])/(x['total_words_after_cleaning'] + 0.000001), 
       axis = 1)

### 1.4 Sentiment score categorization

In [33]:
def sentiment_score_categorization(x):
    if x <= -0.5:
        return "most_negative"
    elif x > -0.5 and x < 0:
        return "negative"
    elif x == 0:
        return "negative"
    elif x > 0 and x < 0.5:
        return "positive"
    else:
        return "very_positive"

In [34]:
cik_list['sentiment_score_categorization'] = cik_list.polarity_score.apply(sentiment_score_categorization)

In [35]:
cik_list.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,total_words_after_cleaning,polarity_score,subjectivity_score,sentiment_score_categorization
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,6.0,39.0,126872.0,-0.733333,0.000355,most_negative
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,3.0,34.0,79339.0,-0.837838,0.000466,most_negative
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,1.0,881.0,-0.999999,0.001135,most_negative
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,5.0,32.0,69772.0,-0.729730,0.000530,most_negative
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,1.0,1072.0,-0.999999,0.000933,most_negative


## 2. Analysis of Readability - Gunning Fox index

In [36]:
# getting number of sentences
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(sent_tokenize(f))

    # total words after cleaning
    cik_list.loc[i, 'total_sentences_after_cleaning'] = sum(doc_dict.values())

In [37]:
cik_list['average_sentence_length'] = cik_list\
.apply(lambda x: x['total_words_after_cleaning']/x['total_sentences_after_cleaning'], axis = 1)

### Getting complex words

Anything word with more than two syllables

In [38]:
dic = pyphen.Pyphen(lang='en')

In [39]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    complex_words = (1 for word in doc_dict if len(dic.inserted(word)) > 2)
    cik_list.loc[i, 'complex_word_count'] = sum(complex_words)

In [40]:
cik_list['percentage_of_complex_words'] = cik_list['complex_word_count']/cik_list['total_words_after_cleaning']

In [41]:
cik_list['fog_index'] = 0.4 * (cik_list['average_sentence_length'] + cik_list['percentage_of_complex_words'])

In [42]:
cik_list.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,total_words_after_cleaning,polarity_score,subjectivity_score,sentiment_score_categorization,total_sentences_after_cleaning,average_sentence_length,complex_word_count,percentage_of_complex_words,fog_index
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,6.0,39.0,126872.0,-0.733333,0.000355,most_negative,3975.0,31.917484,7889.0,0.062181,12.791866
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,3.0,34.0,79339.0,-0.837838,0.000466,most_negative,2499.0,31.748299,5880.0,0.074112,12.728965
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,1.0,881.0,-0.999999,0.001135,most_negative,20.0,44.050000,334.0,0.379115,17.771646
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,5.0,32.0,69772.0,-0.729730,0.000530,most_negative,2487.0,28.054684,6833.0,0.097933,11.261047
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,1.0,1072.0,-0.999999,0.000933,most_negative,25.0,42.880000,358.0,0.333955,17.285582


## 5. Word Count

In [43]:
cachedStopWords = nltk_stopwords.words("english")
stopwords_dict_nltk = Counter(cachedStopWords)
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
punctuation_dict = Counter(punctuations)

In [44]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # removing stop words and counting words
    word_count = (1 for word in doc_dict if word not in stopwords_dict_nltk and word not in punctuation_dict)
    cik_list.loc[i, 'word_count'] = sum(word_count)

In [45]:
cik_list.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,total_words_after_cleaning,polarity_score,subjectivity_score,sentiment_score_categorization,total_sentences_after_cleaning,average_sentence_length,complex_word_count,percentage_of_complex_words,fog_index,word_count
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,6.0,39.0,126872.0,-0.733333,0.000355,most_negative,3975.0,31.917484,7889.0,0.062181,12.791866,8052.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,3.0,34.0,79339.0,-0.837838,0.000466,most_negative,2499.0,31.748299,5880.0,0.074112,12.728965,6031.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,1.0,881.0,-0.999999,0.001135,most_negative,20.0,44.050000,334.0,0.379115,17.771646,358.0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,5.0,32.0,69772.0,-0.729730,0.000530,most_negative,2487.0,28.054684,6833.0,0.097933,11.261047,6981.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,1.0,1072.0,-0.999999,0.000933,most_negative,25.0,42.880000,358.0,0.333955,17.285582,383.0


## Uncertainty and constraining

In [46]:
uncertainty_dictionary = pd.read_excel('data/uncertainty_dictionary.xlsx')
constraining_dictionary = pd.read_excel('data/constraining_dictionary.xlsx')
uncertainty_dict = Counter(uncertainty_dictionary['Word'].to_list())
constraining_dict = Counter(constraining_dictionary['Word'].to_list())

In [47]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting uncertainity
    uncertainty_f_dict = doc_dict & uncertainty_dict
    cik_list.loc[i, 'uncertainty'] = sum(uncertainty_f_dict.values())
    
    # getting constraining
    constraining_f_dict = doc_dict & constraining_dict
    cik_list.loc[i, 'constraining'] = sum(constraining_f_dict.values())

In [48]:
cik_list.head(5).T

,0,1,2,3,4
CIK,3662,3662,3662,3662,3662
CONAME,SUNBEAM CORP/FL/,SUNBEAM CORP/FL/,SUNBEAM CORP/FL/,SUNBEAM CORP/FL/,SUNBEAM CORP/FL/
FYRMO,199803,199805,199808,199811,199811
FDATE,1998-03-06 00:00:00,1998-05-15 00:00:00,1998-08-13 00:00:00,1998-11-12 00:00:00,1998-11-16 00:00:00
FORM,10-K405,10-Q,NT 10-Q,10-K/A,NT 10-Q
SECFNAME,edgar/data/3662/0000950170-98-000413.txt,edgar/data/3662/0000950170-98-001001.txt,edgar/data/3662/0000950172-98-000783.txt,edgar/data/3662/0000950170-98-002145.txt,edgar/data/3662/0000950172-98-001203.txt
positive_score,6,3,0,5,0
negative_score,39,34,1,32,1
total_words_after_cleaning,126872,79339,881,69772,1072
polarity_score,-0.733333,-0.837838,-0.999999,-0.72973,-0.999999


## Sections

In [49]:
# dividing each report into its three sections

for i, row in cik_list.iterrows():
    f = open('data/reports/' + row['SECFNAME'].split(os.sep)[3], 'r')
    
    page = 0
    previous_line = ""
    mda_flag = False
    mda = ""
    qqdmr_flag = False
    qqdmr = ""
    rf_flag = False
    rf = ""
    
    for line in f:
        if mda_flag and 'ITEM' in line:
            mda_flag = False
        if mda_flag:
            mda = " ".join((mda, line))
        if qqdmr_flag and 'ITEM' in line:
            qqdmr_flag = False
        if qqdmr_flag:
            qqdmr = " ".join((qqdmr, line))
        if rf_flag and 'ITEM' in line:
            rf_flag = False
        if rf_flag:
            rf = " ".join((rf, line))
        if "PAGE" in line:
            try:
                page = int(re.findall(r'\d+', previous_line)[0])
            except:
                continue
        if "MANAGEMENT'S DISCUSSION AND ANALYSIS" in line and page >= 1:
            mda_flag = True
        if "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK" in line and page >= 1:
            qqdmr_flag = True
        if "RISK FACTORS" in line and page >= 1:
            rf_flag = True
        previous_line = line
    
    open('data/reports/mda/' + row['SECFNAME'].split(os.sep)[3], 'w').write(mda)
    open('data/reports/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'w').write(qqdmr)
    open('data/reports/rf/' + row['SECFNAME'].split(os.sep)[3], 'w').write(rf)

## Getting Scores

In [50]:
cik_list = pd.read_excel('data/cik_list.xlsx')

### MDA

In [51]:
# iterating through each file, removing stop words and saving it

for file in cik_list.SECFNAME:
    f = open('data/reports/mda/' + file.split(os.sep)[3], 'r').read()
    f_clean = ' '.join([word for word in f.split() if word not in stopwords_dict])
    open('data/reports_cleaned/mda/' + file.split(os.sep)[3], 'w').write(f_clean)

In [52]:
# iterating through each file

for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/mda/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting positive score
    positive_f_dict = doc_dict & positive_dict
    cik_list.loc[i, 'mda_positive_score'] = sum(positive_f_dict.values())
    
    # getting negative score
    negative_f_dict = doc_dict & negative_dict
    cik_list.loc[i, 'mda_negative_score'] = sum(negative_f_dict.values())
    
    # total words after cleaning
    cik_list.loc[i, 'mda_total_words_after_cleaning'] = sum(doc_dict.values())

In [53]:
cik_list['mda_polarity_score'] = cik_list\
.apply(lambda x: (x['mda_positive_score'] - x['mda_negative_score'])/((x['mda_positive_score'] \
                                                                       + x['mda_negative_score'])+0.000001),
      axis = 1)

In [54]:
cik_list['mda_subjectivity_score'] = cik_list\
.apply(lambda x: (x['mda_positive_score'] + x['mda_negative_score'])/(x['mda_total_words_after_cleaning'] + 0.000001), 
       axis = 1)

In [55]:
cik_list['mda_sentiment_score_categorization'] = cik_list.mda_polarity_score.apply(sentiment_score_categorization)

In [56]:
# getting number of sentences
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/mda/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(sent_tokenize(f))

    # total words after cleaning
    cik_list.loc[i, 'mda_total_sentences_after_cleaning'] = sum(doc_dict.values())

In [57]:
def get_avg_sent_length(x):
    try:
        return x['mda_total_words_after_cleaning']/x['mda_total_sentences_after_cleaning']
    except:
        return 0

In [58]:
cik_list['mda_average_sentence_length'] = cik_list\
.apply(lambda x: get_avg_sent_length(x), axis = 1)

In [59]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/mda/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    complex_words = (1 for word in doc_dict if len(dic.inserted(word)) > 2)
    cik_list.loc[i, 'mda_complex_word_count'] = sum(complex_words)

In [60]:
cik_list['mda_percentage_of_complex_words'] = cik_list['mda_complex_word_count']/cik_list['mda_total_words_after_cleaning']

In [61]:
cik_list['mda_fog_index'] = 0.4 * (cik_list['mda_average_sentence_length'] + cik_list['mda_percentage_of_complex_words'])

In [62]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/mda/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # removing stop words and counting words
    word_count = (1 for word in doc_dict if word not in stopwords_dict_nltk and word not in punctuation_dict)
    cik_list.loc[i, 'mda_word_count'] = sum(word_count)

In [63]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/mda/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting uncertainity
    uncertainty_f_dict = doc_dict & uncertainty_dict
    cik_list.loc[i, 'mda_uncertainty_score'] = sum(uncertainty_f_dict.values())
    
    # getting constraining
    constraining_f_dict = doc_dict & constraining_dict
    cik_list.loc[i, 'mda_constraining_score'] = sum(constraining_f_dict.values())

### QQDMR

In [64]:
# iterating through each file, removing stop words and saving it

for file in cik_list.SECFNAME:
    f = open('data/reports/qqdmr/' + file.split(os.sep)[3], 'r').read()
    f_clean = ' '.join([word for word in f.split() if word not in stopwords_dict])
    open('data/reports_cleaned/qqdmr/' + file.split(os.sep)[3], 'w').write(f_clean)

In [65]:
# iterating through each file

for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting positive score
    positive_f_dict = doc_dict & positive_dict
    cik_list.loc[i, 'qqdmr_positive_score'] = sum(positive_f_dict.values())
    
    # getting negative score
    negative_f_dict = doc_dict & negative_dict
    cik_list.loc[i, 'qqdmr_negative_score'] = sum(negative_f_dict.values())
    
    # total words after cleaning
    cik_list.loc[i, 'qqdmr_total_words_after_cleaning'] = sum(doc_dict.values())

In [66]:
cik_list['qqdmr_polarity_score'] = cik_list\
.apply(lambda x: (x['qqdmr_positive_score'] - x['qqdmr_negative_score'])/((x['qqdmr_positive_score'] \
                                                                       + x['qqdmr_negative_score'])+0.000001),
      axis = 1)

In [67]:
cik_list['qqdmr_subjectivity_score'] = cik_list\
.apply(lambda x: (x['qqdmr_positive_score'] + x['qqdmr_negative_score'])/(x['qqdmr_total_words_after_cleaning'] + 0.000001), 
       axis = 1)

In [68]:
cik_list['qqdmr_sentiment_score_categorization'] = cik_list.qqdmr_polarity_score.apply(sentiment_score_categorization)

In [69]:
# getting number of sentences
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(sent_tokenize(f))

    # total words after cleaning
    cik_list.loc[i, 'qqdmr_total_sentences_after_cleaning'] = sum(doc_dict.values())

In [70]:
def get_avg_sent_length(x):
    try:
        return x['qqdmr_total_words_after_cleaning']/x['qqdmr_total_sentences_after_cleaning']
    except:
        return 0

In [71]:
cik_list['qqdmr_average_sentence_length'] = cik_list\
.apply(lambda x: get_avg_sent_length(x), axis = 1)

In [72]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    complex_words = (1 for word in doc_dict if len(dic.inserted(word)) > 2)
    cik_list.loc[i, 'qqdmr_complex_word_count'] = sum(complex_words)

In [73]:
cik_list['qqdmr_percentage_of_complex_words'] = cik_list['qqdmr_complex_word_count']/cik_list['qqdmr_total_words_after_cleaning']

In [74]:
cik_list['qqdmr_fog_index'] = 0.4 * (cik_list['qqdmr_average_sentence_length'] + cik_list['qqdmr_percentage_of_complex_words'])

In [75]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # removing stop words and counting words
    word_count = (1 for word in doc_dict if word not in stopwords_dict_nltk and word not in punctuation_dict)
    cik_list.loc[i, 'qqdmr_word_count'] = sum(word_count)

In [76]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/qqdmr/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting uncertainity
    uncertainty_f_dict = doc_dict & uncertainty_dict
    cik_list.loc[i, 'qqdmr_uncertainty_score'] = sum(uncertainty_f_dict.values())
    
    # getting constraining
    constraining_f_dict = doc_dict & constraining_dict
    cik_list.loc[i, 'qqdmr_constraining_score'] = sum(constraining_f_dict.values())

## RF

In [77]:
# iterating through each file, removing stop words and saving it

for file in cik_list.SECFNAME:
    f = open('data/reports/rf/' + file.split(os.sep)[3], 'r').read()
    f_clean = ' '.join([word for word in f.split() if word not in stopwords_dict])
    open('data/reports_cleaned/rf/' + file.split(os.sep)[3], 'w').write(f_clean)

In [78]:
# iterating through each file

for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/rf/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting positive score
    positive_f_dict = doc_dict & positive_dict
    cik_list.loc[i, 'rf_positive_score'] = sum(positive_f_dict.values())
    
    # getting negative score
    negative_f_dict = doc_dict & negative_dict
    cik_list.loc[i, 'rf_negative_score'] = sum(negative_f_dict.values())
    
    # total words after cleaning
    cik_list.loc[i, 'rf_total_words_after_cleaning'] = sum(doc_dict.values())

In [79]:
cik_list['rf_polarity_score'] = cik_list\
.apply(lambda x: (x['rf_positive_score'] - x['rf_negative_score'])/((x['rf_positive_score'] \
                                                                       + x['rf_negative_score'])+0.000001),
      axis = 1)

In [80]:
cik_list['rf_subjectivity_score'] = cik_list\
.apply(lambda x: (x['rf_positive_score'] + x['rf_negative_score'])/(x['rf_total_words_after_cleaning'] + 0.000001), 
       axis = 1)

In [81]:
cik_list['rf_sentiment_score_categorization'] = cik_list.rf_polarity_score.apply(sentiment_score_categorization)

In [82]:
# getting number of sentences
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/rf/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(sent_tokenize(f))

    # total words after cleaning
    cik_list.loc[i, 'rf_total_sentences_after_cleaning'] = sum(doc_dict.values())

In [83]:
def get_avg_sent_length(x):
    try:
        return x['rf_total_words_after_cleaning']/x['rf_total_sentences_after_cleaning']
    except:
        return 0

In [84]:
cik_list['rf_average_sentence_length'] = cik_list\
.apply(lambda x: get_avg_sent_length(x), axis = 1)

In [85]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/rf/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    complex_words = (1 for word in doc_dict if len(dic.inserted(word)) > 2)
    cik_list.loc[i, 'rf_complex_word_count'] = sum(complex_words)

In [86]:
cik_list['rf_percentage_of_complex_words'] = cik_list['rf_complex_word_count']/cik_list['rf_total_words_after_cleaning']

In [87]:
cik_list['rf_fog_index'] = 0.4 * (cik_list['rf_average_sentence_length'] + cik_list['rf_percentage_of_complex_words'])

In [88]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/rf/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # removing stop words and counting words
    word_count = (1 for word in doc_dict if word not in stopwords_dict_nltk and word not in punctuation_dict)
    cik_list.loc[i, 'rf_word_count'] = sum(word_count)

In [89]:
for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/rf/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))
    
    # getting uncertainity
    uncertainty_f_dict = doc_dict & uncertainty_dict
    cik_list.loc[i, 'rf_uncertainty_score'] = sum(uncertainty_f_dict.values())
    
    # getting constraining
    constraining_f_dict = doc_dict & constraining_dict
    cik_list.loc[i, 'rf_constraining_score'] = sum(constraining_f_dict.values())

## Additional Variables: positive/negative and uncertainty/constraining word proportion

In [90]:
# word proportions

cik_list['mda_positive_word_proportion'] = cik_list['mda_positive_score']/cik_list['mda_word_count']
cik_list['mda_negative_word_proportion'] = cik_list['mda_negative_score']/cik_list['mda_word_count']
cik_list['qqdmr_positive_word_proportion'] = cik_list['qqdmr_positive_score']/cik_list['qqdmr_word_count']
cik_list['qqdmr_negative_word_proportion'] = cik_list['qqdmr_negative_score']/cik_list['qqdmr_word_count']
cik_list['rf_positive_word_proportion'] = cik_list['rf_positive_score']/cik_list['rf_word_count']
cik_list['rf_negative_word_proportion'] = cik_list['rf_negative_score']/cik_list['rf_word_count']

cik_list['mda_uncertainty_word_proportion'] = cik_list['mda_uncertainty_score']/cik_list['mda_word_count']
cik_list['mda_constraining_word_proportion'] = cik_list['mda_constraining_score']/cik_list['mda_word_count']
cik_list['qqdmr_uncertainty_word_proportion'] = cik_list['qqdmr_uncertainty_score']/cik_list['qqdmr_word_count']
cik_list['qqdmr_constraining_word_proportion'] = cik_list['qqdmr_constraining_score']/cik_list['qqdmr_word_count']
cik_list['rf_uncertainty_word_proportion'] = cik_list['rf_uncertainty_score']/cik_list['rf_word_count']
cik_list['rf_constraining_word_proportion'] = cik_list['rf_constraining_score']/cik_list['rf_word_count']

## Additional Variable: Constraining words for whole report

In [91]:
# constraining_words_whole_report

for i, row in cik_list.iterrows():
    f = open('data/reports_cleaned/' + row['SECFNAME'].split(os.sep)[3], 'r').read()
    doc_dict = Counter(word_tokenize(f))

    # getting constraining
    constraining_f_dict = doc_dict & constraining_dict
    cik_list.loc[i, 'constraining_words_whole_report'] = sum(constraining_f_dict.values())

## Output Data Structure

In [92]:
# filling blanks with NAs

cik_list = cik_list.fillna(0)

In [93]:
original_columns = list(pd.read_excel('data/cik_list.xlsx').columns)

In [94]:
output_columns = [
    'mda_positive_score',
    'mda_negative_score',
    'mda_polarity_score',
    'mda_average_sentence_length',
    'mda_percentage_of_complex_words',
    'mda_fog_index',
    'mda_complex_word_count',
    'mda_word_count',
    'mda_uncertainty_score',
    'mda_constraining_score',
    'mda_positive_word_proportion',
    'mda_negative_word_proportion',
    'mda_uncertainty_word_proportion',
    'mda_constraining_word_proportion',
    'qqdmr_positive_score',
    'qqdmr_negative_score',
    'qqdmr_polarity_score',
    'qqdmr_average_sentence_length',
    'qqdmr_percentage_of_complex_words',
    'qqdmr_fog_index',
    'qqdmr_complex_word_count',
    'qqdmr_word_count',
    'qqdmr_uncertainty_score',
    'qqdmr_constraining_score',
    'qqdmr_positive_word_proportion',
    'qqdmr_negative_word_proportion',
    'qqdmr_uncertainty_word_proportion',
    'qqdmr_constraining_word_proportion',
    'rf_positive_score',
    'rf_negative_score',
    'rf_polarity_score',
    'rf_average_sentence_length',
    'rf_percentage_of_complex_words',
    'rf_fog_index',
    'rf_complex_word_count',
    'rf_word_count',
    'rf_uncertainty_score',
    'rf_constraining_score',
    'rf_positive_word_proportion',
    'rf_negative_word_proportion',
    'rf_uncertainty_word_proportion',
    'rf_constraining_word_proportion',
    'constraining_words_whole_report'
]

In [95]:
keep_columns = original_columns + output_columns

In [96]:
cik_list.columns.intersection(keep_columns)

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'SECFNAME',
       'mda_positive_score', 'mda_negative_score', 'mda_polarity_score',
       'mda_average_sentence_length', 'mda_complex_word_count',
       'mda_percentage_of_complex_words', 'mda_fog_index', 'mda_word_count',
       'mda_uncertainty_score', 'mda_constraining_score',
       'qqdmr_positive_score', 'qqdmr_negative_score', 'qqdmr_polarity_score',
       'qqdmr_average_sentence_length', 'qqdmr_complex_word_count',
       'qqdmr_percentage_of_complex_words', 'qqdmr_fog_index',
       'qqdmr_word_count', 'qqdmr_uncertainty_score',
       'qqdmr_constraining_score', 'rf_positive_score', 'rf_negative_score',
       'rf_polarity_score', 'rf_average_sentence_length',
       'rf_complex_word_count', 'rf_percentage_of_complex_words',
       'rf_fog_index', 'rf_word_count', 'rf_uncertainty_score',
       'rf_constraining_score', 'mda_positive_word_proportion',
       'mda_negative_word_proportion', 'qqdmr_positive_word_proportion',


In [99]:
cik_list = cik_list[cik_list.columns.intersection(keep_columns)]

In [100]:
cik_list.to_csv('out_data_structure.csv', index = False)